<a href="https://colab.research.google.com/github/sadrabr/LLM_projects/blob/main/Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install requirement
!pip install python-dotenv

### import

In [ ]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [ ]:
# load_dotenv(override=True)
# api_key = os.getenv('OPENAI_API_KEY')

# # Check the key

# if not api_key:
#     print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
# elif not api_key.startswith("sk-proj-"):
#     print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
# elif api_key.strip() != api_key:
#     print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
# else:
#     print("API key found and looks good so far!")

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
# Start Ollama server

import subprocess

# Start the command in a background process
process = subprocess.Popen(["ollama", "serve"])

# The kernel can continue execution while the process runs in the background
print("The 'ollama serve' process is running in the background.")

The 'ollama serve' process is running in the background.


In [ ]:
# Download the model that we want to use - this might take a couple of mins
# The exclamation mark at the start of the line indicates that this is a shell command, not python

!ollama pull llama3.2:1b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest 
pulling 74701a8c35f6:   0% ▕▏ 4.7 MB/1.3 GB                  pulling manifest 
pulling 74701a8c35f6:   3% ▕▏  39 MB/1.3 GB                  pulling manifest 
pulling 74701a8c35f6:   4% ▕▏  50 MB/1.3 GB                  pulling manifest 
pulling 74701a8c35f6:   6% ▕▏  75 MB/1.3 GB                  pulling manifest 
pulling 74701a8c35f6:   9% ▕▏ 119 MB/1.3 GB                  pulling manifest 
pulling 74701a8c35f6:  10% ▕▏ 132 MB/1.3 GB                  pulling manifest 
pulling 74701a8c35f6:  13% ▕▏ 165 MB/1.3 GB                  pulling manifest 
pulling 74701a8c35f6:  15% ▕▏ 198 MB/1.3 GB                  pulling manifest 
pulling 74701a8c35f6:  17% ▕▏ 219 MB/1.3 GB                  pulling manifest 
pulling 74701a8c35f6:  19% ▕▏ 251 MB/1.3 GB                  pulling manifest 
pu

In [ ]:
# install the Ollama python API package

!pip install ollama

In [ ]:
# Your first message to your very own LLM running "locally" (well, on your cloud box)

message = "Hello,  This is my first ever message to you! Hi!"

In [ ]:
# Put this message into the format expected by Ollama

messages = [{"role": "user", "content": message}]

In [ ]:
# import the package

import ollama

In [ ]:
# Let's do this!!

response = ollama.chat(model="llama3.2:1b", messages=messages)

print(response['message']['content'])

Welcome to our conversation! I'm glad to meet you. Is there something I can help you with or would you like to chat?


In [ ]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

In [ ]:
# A class to represent a Webpage
class Website:

  def __init__(self, url):
     """
        Create this Website object from the given url using the BeautifulSoup library
     """
     self.url = url
     response = requests.get(url, headers=headers)
     soup = BeautifulSoup(response.content, 'html.parser')
     self.title = soup.title.string if soup.title else "No title found"
     for irrelevant in soup.body(["script", "style", "img", "input"]):
      irrelevant.decompose()
     self.text = soup.body.get_text(separator="\n",strip=True)

In [ ]:
ed = Website("https://edwarddonner.com")
print(ed.title)
print(ed.text)

In [ ]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [ ]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [ ]:
user_prompt_for(ed)

'You are looking at a website titled Home - Edward Donner\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nHome\nConnect Four\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI sta

In [ ]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]


In [ ]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = ollama.chat(model="llama3.2:1b", messages=messages_for(website))
    return response['message']['content']


In [ ]:
summarize("https://edwarddonner.com")

'# Edward Donner\'s Website Summary\n\n* **Website Overview**: The website is about a team called Nebula.io, which provides a platform for LLMs (Large Language Models) to assist with talent discovery and management.\n* **Founders and Background**: The founder and CTO of Nebula.io is Ed, who previously founded an AI startup untapt that was acquired in 2021. He has patents and press coverage related to his work on matching models and proprietary LLMs.\n* **Courses and Resources**: The website lists several courses and resources available for Agentic AI Engineering, including:\n\t+ "The Complete Agentic AI Engineering Course" (announced)\n\t+ "LLM Workshop – Hands-on with Agents – resources" (available in December 2024)\n\t+ "Welcome, SuperDataScientists!" (announced)\n* **Contact Information**: The website has a contact form, email address, LinkedIn profile, Twitter handle, Facebook page, and newsletter sign-up link.'

In [ ]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [ ]:
display_summary("https://edwarddonner.com")

Here is a summary of the website in markdown:

* The website is about an LLM arena game where LLMs (Large Language Models) compete against each other.
* It appears to be a platform for recruiting and managing talent using AI technology.
* The founder, Ed Donner, has a background in AI engineering and previously founded an AI startup called untapt, which was acquired in 2021.
* The website features resources and announcements related to the AI industry, including upcoming courses and workshops.

In [ ]:
display_summary("https://cnn.com")

In [ ]:
display_summary("https://theconversation.com/the-womens-health-initiative-has-shaped-womens-health-for-over-30-years-but-its-future-is-uncertain-255311")

**The Women's Health Initiative: Shaping Women's Health for Over 30 Years**

* The Women's Health Initiative (WHI) is a long-term study of women's health initiated by the National Institutes of Health (NIH) in 1991.
* The study aims to understand the risks and benefits of menopausal hormone therapy, with over 2,400 published scientific papers in leading medical journals.
* Despite its importance, the WHI has faced funding uncertainty, with plans for termination being reinstated after controversy.
* The study has had a significant impact on women's health, preventing thousands of cases of breast cancer and cardiovascular disease, and reducing healthcare costs by an estimated $35 billion.

**Current Status**

* Almost 42,000 women remain active participants in the WHI, representing over 50% of the population.
* The study continues to collect data from these participants, providing valuable insights into health outcomes related to age-related diseases.
* New research is being conducted on topics such as physical activity, bone health, and disease prevention.

**Future Directions**

* Over 42,000 additional women are expected to join the study by 2050.
* The WHI's findings will continue to guide clinical practice, prevention strategies, and public health policies, particularly for older women.
* Researchers are exploring new avenues of research, including the effects of low to moderate physical activity on health and bone health.

**Impact and Legacy**

* The Women's Health Initiative has shaped women's health in a significant way, providing valuable insights into the risks and benefits of menopausal hormone therapy.
* Its findings have been instrumental in preventing thousands of cases of breast cancer and cardiovascular disease, reducing healthcare costs by an estimated $35 billion.